In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
X = df.drop('total_bill', axis =1)
y = df['total_bill']

In [6]:
X

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,5.92,Male,No,Sat,Dinner,3
240,2.00,Female,Yes,Sat,Dinner,2
241,2.00,Male,Yes,Sat,Dinner,2
242,1.75,Male,No,Sat,Dinner,2


In [7]:
y

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [9]:
X_train.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
154,2.00,Male,No,Sun,Dinner,4
167,4.50,Male,No,Sun,Dinner,4
110,3.00,Male,No,Sat,Dinner,2
225,2.50,Female,Yes,Fri,Lunch,2


In [10]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [11]:
#Handling the missing value

from sklearn.impute import SimpleImputer #For missing values
from sklearn.preprocessing import OneHotEncoder #For Encoding
from sklearn.preprocessing import StandardScaler #For Scaling

from sklearn.pipeline import Pipeline #A sequence of data transformer
from sklearn.compose import ColumnTransformer # Groups all the pipeline steps for each column

In [16]:
cat_cols = ['sex', 'smoker', 'day', 'time']
num_cols = ['tip', 'size']

In [17]:
num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = 'median')),
                                ('sclaing', StandardScaler())])
cat_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = 'most_frequent')),
                                ('encoding', OneHotEncoder())])

In [18]:
preprocessor = ColumnTransformer([('num_pipeline', num_pipeline, num_cols), 
                                  ('cat_pipeline', cat_pipeline, cat_cols)])


In [19]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [20]:
X_train

array([[-1.47443803, -0.57766863,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.71612531,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.19880579,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.43283335, -0.57766863,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.29060464, -0.57766863,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.1034652 ,  0.44638031,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [21]:

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [22]:
models = {'Support Vector Regressor' : SVR(),
         'DT Regressor' : DecisionTreeRegressor(),
         'Multiple Linear Regression' : LinearRegression(),
         'Random Forest Regressor' : RandomForestRegressor()}

In [23]:
from sklearn.metrics import r2_score
def model_train_eval(X_train, y_train, X_test, y_test, models):
    evaluation = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_score = r2_score(y_test, y_pred)
        evaluation[list(models.keys())[i]] = model_score
    return evaluation

In [24]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'Support Vector Regressor': 0.4069433055293954,
 'DT Regressor': 0.1547931689351666,
 'Multiple Linear Regression': 0.6719006761625261,
 'Random Forest Regressor': 0.6245349538276963}

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
rfr = RandomForestRegressor(oob_score = True)

In [27]:
rfr

RandomForestRegressor(oob_score=True)

In [28]:
from sklearn.model_selection import RandomizedSearchCV
params = {'max_depth' : [1, 50, 100, 150, 200],
         'n_estimators' : [50, 100, 200]}

In [29]:
params

{'max_depth': [1, 50, 100, 150, 200], 'n_estimators': [50, 100, 200]}

In [31]:
reg = RandomizedSearchCV(rfr, param_distributions = params, cv = 5, verbose = 3, scoring = 'r2')

In [32]:
reg

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=3)

In [33]:
reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ....max_depth=50, n_estimators=100;, score=0.262 total time=   0.4s
[CV 2/5] END ....max_depth=50, n_estimators=100;, score=0.472 total time=   0.3s
[CV 3/5] END ....max_depth=50, n_estimators=100;, score=0.259 total time=   0.3s
[CV 4/5] END ....max_depth=50, n_estimators=100;, score=0.590 total time=   0.3s
[CV 5/5] END ....max_depth=50, n_estimators=100;, score=0.361 total time=   0.2s
[CV 1/5] END ....max_depth=100, n_estimators=50;, score=0.269 total time=   0.1s
[CV 2/5] END ....max_depth=100, n_estimators=50;, score=0.499 total time=   0.1s
[CV 3/5] END ....max_depth=100, n_estimators=50;, score=0.137 total time=   0.1s
[CV 4/5] END ....max_depth=100, n_estimators=50;, score=0.624 total time=   0.1s
[CV 5/5] END ....max_depth=100, n_estimators=50;, score=0.356 total time=   0.1s
[CV 1/5] END ...max_depth=200, n_estimators=200;, score=0.315 total time=   0.7s
[CV 2/5] END ...max_depth=200, n_estimators=200;

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=3)

In [34]:
reg.best_params_

{'n_estimators': 100, 'max_depth': 200}

In [35]:
reg.best_score_

0.41365241806000375